# Create our own AI Research Assistant with PUCP's Economics thesis repository

The main propouse of our code is develop a AI Research Assistant the help us finding thesis from [PUCP's Economics repository](https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/1026). This development help us to the research of main findings related to the Public Policy in Economics using a largest public repository with OPEN-AI.

## Install packages

In [135]:
!pip install openai tiktoken

### Set up our API KEY

In [4]:
import getpass, openai, os
client = getpass.getpass(prompt="OPENAI - KEY: ")
openai.apikey = client
os.environ["OPENAI_API_KEY"] = client

OPENAI - KEY: ··········


### Call to OpenAI

In [5]:
from openai import OpenAI
client = OpenAI()

### Web scrap PUCP's Economics thesis repository

#### Import our package

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

#### Applied wep scraping to all thesis with their abstracts from the first 4 pages

In [6]:
# URL base de la página
base_url = "https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/1026/recent-submissions?offset="

# Lista para almacenar los datos
data = []

# Función para obtener el resumen dado un enlace
def obtener_resumen(enlace):
    # Hacer la solicitud GET al enlace
    response = requests.get(enlace)

    # Parsear el contenido HTML utilizando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar el elemento que contiene el resumen
    resumen_elemento = soup.find('div', class_='simple-item-view-description')

    # Si se encontró el elemento del resumen
    if resumen_elemento:
        # Obtener el texto del resumen y retornarlo
        return resumen_elemento.text.strip()
    else:
        return None

# Iterar sobre los enlaces
for offset in range(0, 60, 20):
    # Construir la URL completa
    url = base_url + str(offset)

    # Hacer la solicitud GET
    response = requests.get(url)

    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todos los elementos <li> que contienen los títulos y enlaces
    items = soup.find_all('li')

    # Iterar sobre los elementos encontrados
    for item in items:
        # Encontrar el elemento <h4> dentro del <li>
        titulo_elemento = item.find('h4')

        # Si el elemento <h4> existe
        if titulo_elemento:
            # Obtener el texto del título
            titulo = titulo_elemento.text.strip()

            # Encontrar el enlace dentro del <h4>
            enlace_elemento = titulo_elemento.find('a')

            # Si el enlace existe
            if enlace_elemento:
                # Obtener la URL del enlace
                enlace = "https://tesis.pucp.edu.pe" + enlace_elemento['href']

                # Obtener el resumen del enlace
                resumen = obtener_resumen(enlace)

                # Si se obtuvo el resumen, agregar el título y resumen al diccionario
                if resumen:
                    data.append({"title": titulo, "abstract": resumen})

# Convertir la lista a JSON
json_data = json.dumps(data, ensure_ascii=False, indent=4)

# Imprimir el JSON
print(json_data)


[
    {
        "title": "La supervisión del sistema bancario en el marco de la protección al consumidor\n                    ﻿",
        "abstract": "Abstract\nEn el presente Trabajo de Suficiencia Profesional expondré mi experiencia\r\nprofesional como asistente en temas económicos en el Instituto Nacional de Defensa\r\nde la Competencia y de la Protección de la Propiedad Intelectual, en adelante\r\nINDECOPI. Específicamente en el área encargada de la supervisión al sistema\r\nfinanciero regulado dentro de la Gerencia de Supervisión y Fiscalización. La gerencia\r\ninterviene en casos de impacto nacional y en sectores sensibles del mercado con el\r\nfin de disuadir conductas infractoras.\r\nEl documento expone el marco normativo bajo el que se rigen las supervisiones\r\nbancarias, define las conductas infractoras bajo supervisión, detalla la metodología\r\nde las supervisiones y realiza un ejemplo de supervisión a una entidad financiera, el\r\nmismo que permite ver las infracciones má

## Set up our AI Research Assistant and the parameters of the final public policy findings abstracts of a specific topic

In [132]:
DELIMITER = "####"
# Set up our completion messages
def getCompletionFromMessages(
        query,
        messages,
        model = "gpt-4",
        temperature = 0,
        delimiter = DELIMITER
):
    query = f"{DELIMITER}{query}{DELIMITER}"
    messages += [{"role": "user", "content": query}]
    response = client.chat.completions.create(
        messages = messages,
        temperature = temperature,
        model = model
    )
    responseContent = response.choices[0].message.content
    messages += [{"content": responseContent, "role": "assistant"}]
    return messages


# Create summarised abstracts
def getCompletionFromThesisData(query, messages, data, delimiter="####"):
    processedQuery = f"{delimiter}{query}{delimiter}"
    # Search the topic in the title or abstracts
    relevant_thesis = [thesis for thesis in data if query.lower() in thesis['title'].lower() or query.lower() in thesis['abstract'].lower()]
    abstracts = " ".join([thesis['abstract'] for thesis in relevant_thesis]) if relevant_thesis else "No relevant thesis found."
    # Set up role instructions
    role =  [{'role':'system', 'content': "you are a expert economist that works related to public policy - use all the information from academics articles provided to summarise all the abstracts in one original paragrah with the main findings of the the topic, return all in spanish and with the maxium 400 words"}]
    prueba = getCompletionFromMessages(abstracts, role)
    # Save the final summary
    prueba = prueba[-1]["content"]

    processedQuery += f"\n{prueba}"
    messages = processedQuery
    return messages

## Pilot of AI Research Assistant with topic 'empleo'

In [133]:
# Crear una lista vacía para almacenar los mensajes
messages = []

# Topic
user_query = "empleo"

# Llamar a la función para obtener los datos relevantes de la tesis
messages = getCompletionFromThesisData(user_query, messages, data)


## Muestra de resumen

In [134]:
messages

'####empleo####\nEste resumen aborda tres estudios económicos relacionados con la política pública. El primer estudio examina el impacto de los cambios en el salario mínimo en la duración del empleo en Lima Metropolitana, utilizando una metodología de análisis de supervivencia. Los resultados indican que los salarios cercanos al salario mínimo tienen un efecto significativo, aunque este se ve opacado en micro o pequeñas empresas, o en individuos sin estudios superiores. El segundo estudio se centra en el papel del Consejo Nacional de Trabajo y Promoción del Empleo (CNTPE) en la regulación de las remuneraciones mínimas en Perú, destacando la experiencia laboral del autor como especialista en economía en la Comisión Especial de Productividad y Salarios Mínimos (CEPSM) del CNTPE. Finalmente, el tercer estudio analiza la desigualdad de ingresos en Perú entre 2012 y 2020, utilizando regresiones Recentered Influence Function (RIF) y un método de descomposición según fuente de ingresos. El es